# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models. It uses Minikube as the target Kubernetes cluster.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)

Start minikube and ensure custom resource validation is activated and ther is 5G of memory.

In [ ]:
!minikube start --memory=5000 --feature-gates=CustomResourceValidation=true

Install Helm

In [1]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


Label the node to allow load testing to run on it

In [1]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "fresh" not labeled


## Set up REST and gRPC methods

Install gRPC modules for the prediction protos.

In [2]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I./proto --python_out=./proto --grpc_python_out=./proto ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

In [3]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
import commands

MINIKUBE_IP=commands.getoutput('minikube ip')

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://"+MINIKUBE_IP+":30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel(MINIKUBE_IP+":30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [7]:
!helm install ../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

NAME:   seldon-core
LAST DEPLOYED: Wed Jan 24 16:31:43 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           2s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          2s
seldon-apiserver         1        1        1           0          2s
seldon-cluster-manager   1        1        1           0          2s
grafana-prom-deployment  1        1        1           0          2s
kafka                    1        1        1           0          2s
prometheus-deployment    1        1        1           0          2s
redis                    1        1        1           0          2s

==> v1/Service
NAME                      TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
alertmanager              ClusterIP  10.109.60.246   <none>       80/TCP  

Check all services are running before proceeding.

In [6]:
!kubectl get pods

NAME                                       READY     STATUS              RESTARTS   AGE
alertmanager-deployment-6f5c9c5f58-hzfvs   0/1       ContainerCreating   0          3s
grafana-prom-deployment-bf46c5988-89744    0/1       ContainerCreating   0          3s
grafana-prom-import-dashboards-c7gbj       0/1       ContainerCreating   0          3s
kafka-68cc697c5-hhdvs                      0/1       ContainerCreating   0          3s
prometheus-deployment-6b7f9fcbc6-f56n9     0/1       ContainerCreating   0          3s
prometheus-node-exporter-ddlkl             0/1       ContainerCreating   0          3s
redis-5767447797-h456r                     0/1       ContainerCreating   0          3s
seldon-apiserver-775c6f4cdf-8hzps          1/1       Running             0          3s
seldon-cluster-manager-69fcb5f4-2vt2p      1/1       Running             0          3s
zookeeper-1                                0/1       ContainerCreating   0          2s
zookeeper-2                               

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [7]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"apiVersion":"machinelearning.seldon.io/v1alpha1", "kind":"SeldonDeployment", "metadata":map[string]interface {}{"clusterName":"", "namespace":"default", "deletionTimestamp":interface {}(nil), "creationTimestamp":"2018-01-09T16:42:10Z", "uid":"09e309f1-f55c-11e7-94f2-0800274e805e", "selfLink":"", "initializers":interface {}(nil), "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "deletionGracePeriodSeconds":(*int64)(nil)}, "spec":map[string]interface {}{"annotations":map[string]interface {}{"deployment_version":"v1", "project_name":"FX Market Prediction"}, "name":"test-deployment", "oauth_key":1234, "oauth_secret":"oauth-secret", "predictors":[]interface {}{map[string]interface {}{"name":"fx-market-predictor", "replicas":1, "annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]int

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [ ]:
!kubectl create -f resources/model_invalid2.json

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

In [ ]:
!kubectl delete -f resources/model_invalid2.json

## Normal Operation
A simple example is shown below we use a single prepacked model for illustration. The spec contains a set of predictors each of which contains a ***componentSpec*** which is a Kubernetes [PodTemplateSpec](https://kubernetes.io/docs/api-reference/v1.9/#podtemplatespec-v1-core) alongside a ***graph*** which describes how components fit together.

In [8]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
       

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [9]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [4]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   1m


In [5]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2018-01-10T10:18:22Z
  Generation:          0
  Initializers:        <nil>
  Resource Version:    10347
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 9670910d-f5ef-11e7-a1a3-0800274e805e
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Name:                    test-deployment
  Oauth _ Key:             oauth-key
  Oauth _ Secret:          oauth-secret
  Predictors:
    Annotations:
      Predictor _

Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [13]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1]]]

## Get predictions

#### REST Request

In [14]:
rest_request()

{
  "meta": {
    "puid": "9ciu7ike3lsus1omus1u4ef82p",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [15]:
grpc_request()

meta {
  puid: "i5pjfu2kerin2of27ecu1m6q5"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Update deployment with canary

We will change the deployment to add a "canary" deployment. This illustrates:
 - Updating a deployment with no downtime
 - Adding an extra predictor to run alongside th exsting predictor.
 
 You could manage different traffic levels by controlling the number of replicas of each.

In [ ]:
!cat resources/model_with_canary.json

In [ ]:
!kubectl apply -f resources/model_with_canary.json

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

#### REST Request

In [ ]:
rest_request()

#### gRPC request

In [ ]:
grpc_request()

## Load test

Start a load test which will post REST requests at 10 requests per second.

!helm install seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --repo https://storage.googleapis.com/seldon-charts

You can view an analytics dashboard inside the cluster at http://192.168.99.100:30034/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:
 - Username : admin
 - password : password (as set when starting seldon-core above)
 
 The dashboard should look like below:
 
 
 <img src="images/dashboard.png" alt="predictor with canary" title="ml graph"/>

# Tear down

In [ ]:
!helm delete loadtest --purge

In [12]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [6]:
!helm delete seldon-core --purge

release "seldon-core" deleted
